<a href="https://colab.research.google.com/github/JustJazz291/First-Repo/blob/main/final_task%201_BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
ba=pd.read_csv('/content/British Airways Summer Schedule Dataset - Forage Data Science Task 1.csv')

In [ ]:
ba.head()

,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,02-09-2025,14:19,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,10-06-2025,06:42,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,27-10-2025,15:33,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,15-06-2025,18:29,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,25-08-2025,20:35,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27


In [ ]:
ba.count()

,0
FLIGHT_DATE,10000
FLIGHT_TIME,10000
TIME_OF_DAY,10000
AIRLINE_CD,10000
FLIGHT_NO,10000
DEPARTURE_STATION_CD,10000
ARRIVAL_STATION_CD,10000
ARRIVAL_COUNTRY,10000
ARRIVAL_REGION,10000
HAUL,10000


In [ ]:
def classify_time_of_day(time):
    if time == 'Morning':
        return 'Early Morning'
    elif time == 'Afternoon':
        return 'Mid-Day'
    elif time == 'Evening':
        return 'Evening'
    else:
        return 'Unknown'

ba['Time_Grouping'] = ba['TIME_OF_DAY'].apply(classify_time_of_day)

In [ ]:
short_haul = ['CDG', 'FRA', 'MAD', 'AMS', 'IST']  # Examples for Europe
long_haul_asia = ['DEL', 'BOM', 'HKG', 'SIN', 'NRT']
north_america = ['JFK', 'LAX', 'ORD', 'SFO', 'YYZ']
club_only = ['EDI', 'GLA', 'MAN']  # Just examples, customize as needed

def classify_region(station):
    if station in short_haul:
        return 'Short-Haul Europe'
    elif station in long_haul_asia:
        return 'Long-Haul Asia'
    elif station in north_america:
        return 'North America'
    elif station in club_only:
        return 'Club-Only Eligible'
    else:
        return 'Other'

ba['Region_Grouping'] = ba['ARRIVAL_STATION_CD'].apply(classify_region)

In [ ]:
def final_grouping(row):
    if row['Region_Grouping'] != 'Other':
        return row['Region_Grouping']
    else:
        return row['Time_Grouping']

ba['Grouping'] = ba.apply(final_grouping, axis=1)

In [ ]:
# Define lounge eligibility mapping
eligibility_map = {
    'Early Morning': 'Not Eligible',
    'Mid-Day': 'Eligible',
    'Evening': 'Eligible',
    'Short-Haul Europe': 'Eligible',
    'Long-Haul Asia': 'Eligible',
    'North America': 'Eligible',
    'Club-Only Eligible': 'Club Members Only'
}

# Add a new column for lounge eligibility
ba['Lounge Eligibility'] = ba['Grouping'].map(eligibility_map)

In [ ]:
comments_map = {
    'Mid-Day': 'Fast Track Available',
    'Club-Only Eligible': 'Based on Club Access'
}

ba['Comments'] = ba['Grouping'].map(comments_map)

In [ ]:
ba[['FLIGHT_DATE', 'FLIGHT_TIME', 'FLIGHT_NO', 'ARRIVAL_STATION_CD', 'Grouping', 'Lounge Eligibility', 'Comments']].head(10)

,FLIGHT_DATE,FLIGHT_TIME,FLIGHT_NO,ARRIVAL_STATION_CD,Grouping,Lounge Eligibility,Comments
0,02-09-2025,14:19,BA5211,LAX,North America,Eligible,NaN
1,10-06-2025,06:42,BA7282,LAX,North America,Eligible,NaN
2,27-10-2025,15:33,BA1896,FRA,Short-Haul Europe,Eligible,NaN
3,15-06-2025,18:29,BA5497,IST,Short-Haul Europe,Eligible,NaN
4,25-08-2025,20:35,BA1493,FRA,Short-Haul Europe,Eligible,NaN
5,12-07-2025,19:08,BA4954,VIE,Evening,Eligible,NaN
6,16-09-2025,21:44,BA9840,AMS,Short-Haul Europe,Eligible,NaN
7,24-04-2025,14:50,BA7116,ORD,North America,Eligible,NaN
8,17-05-2025,12:13,BA4424,FRA,Short-Haul Europe,Eligible,NaN
9,01-05-2025,09:55,BA4878,IST,Short-Haul Europe,Eligible,NaN


In [ ]:
# Count number of each Tier within every Grouping
tier_counts = ba.groupby('Grouping')[['TIER1_ELIGIBLE_PAX', 'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX']].sum()

# Get total per Grouping
tier_totals = tier_counts.sum(axis=1)

# Convert to % per Grouping
tier_percentages = (tier_counts.T / tier_totals).T * 100
tier_percentages = tier_percentages.round(2).reset_index()

# Rename columns for final Excel view
tier_percentages.columns = ['Grouping', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %']
tier_percentages

,Grouping,Tier 1 %,Tier 2 %,Tier 3 %
0,Early Morning,1.54,20.36,78.11
1,Evening,1.55,20.47,77.98
2,Mid-Day,1.57,20.39,78.04
3,North America,1.49,20.43,78.08
4,Short-Haul Europe,1.60,20.38,78.02
5,Unknown,1.66,20.35,78.00


In [ ]:
example_destinations = {
    'Early Morning': 'LHR, AMS',
    'Mid-Day': 'JFK, CDG',
    'Evening': 'DXB, FRA',
    'Short-Haul Europe': 'CDG, MAD, AMS',
    'Long-Haul Asia': 'DEL, BOM, HKG',
    'North America': 'JFK, ORD',
    'Club-Only Eligible': 'MAN, DUB'
}

notes = {
    'Club-Only Eligible': 'Club members only',
    'Mid-Day': 'Includes fast-track lanes'
}

# Map to new columns
tier_percentages['Example Destinations'] = tier_percentages['Grouping'].map(example_destinations)
tier_percentages['Notes'] = tier_percentages['Grouping'].map(notes)

In [ ]:
# Reorder columns
final_lookup_table = tier_percentages[['Grouping', 'Example Destinations', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %', 'Notes']]
import pandas as pd
# import caas_jupyter_tools as tools
# tools.display_dataframe_to_user(name="British Airways Lounge Eligibility Lookup Table", dataframe=final_lookup_table)
display(final_lookup_table)

,Grouping,Example Destinations,Tier 1 %,Tier 2 %,Tier 3 %,Notes
0,Early Morning,"LHR, AMS",1.54,20.36,78.11,NaN
1,Evening,"DXB, FRA",1.55,20.47,77.98,NaN
2,Mid-Day,"JFK, CDG",1.57,20.39,78.04,Includes fast-track lanes
3,North America,"JFK, ORD",1.49,20.43,78.08,NaN
4,Short-Haul Europe,"CDG, MAD, AMS",1.60,20.38,78.02,NaN
5,Unknown,NaN,1.66,20.35,78.00,NaN


In [20]:
# Count number of each Tier within every Grouping
tier_counts = ba.groupby('Grouping')[['TIER1_ELIGIBLE_PAX', 'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX']].sum()

# Get total per Grouping
tier_totals = tier_counts.sum(axis=1)

# Convert to % per Grouping
tier_percentages = (tier_counts.T / tier_totals).T * 100
tier_percentages = tier_percentages.round(2).reset_index()

# Rename columns for final Excel view
tier_percentages.columns = ['Grouping', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %']

In [22]:
example_dest = (
    ba.groupby('Grouping')['ARRIVAL_STATION_CD']
    .apply(lambda x: ', '.join(x.unique()[:3]))  # Pick top 3 unique destinations
    .reset_index(name='Example Destinations')
)

In [23]:
lookup_table = pd.merge(tier_percentages, example_dest, on='Grouping', how='left')

# Optional: Add Notes manually
lookup_table['Notes'] = lookup_table['Grouping'].map({
    'Club-Only Eligible': 'Only Club Members Allowed',
    'Short-Haul Europe': 'Flights < 1500km within Europe',
    'Long-Haul Asia': 'Asia flights > 3500km'
    # Add more notes as needed
})

In [25]:
# Reorder columns
lookup_table = lookup_table[['Grouping', 'Example Destinations', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %', 'Notes']]
import pandas as pd
# import caas_jupyter_tools as tools
# tools.display_dataframe_to_user(name="Lounge Eligibility Lookup (by Grouping)", dataframe=lookup_table)
display(lookup_table)

,Grouping,Example Destinations,Tier 1 %,Tier 2 %,Tier 3 %,Notes
0,Early Morning,"MUC, HND, ZRH",1.54,20.36,78.11,NaN
1,Evening,"VIE, MUC, DXB",1.55,20.47,77.98,NaN
2,Mid-Day,"VIE, DXB, MUC",1.57,20.39,78.04,NaN
3,North America,"LAX, ORD, JFK",1.49,20.43,78.08,NaN
4,Short-Haul Europe,"FRA, IST, AMS",1.60,20.38,78.02,Flights < 1500km within Europe
5,Unknown,"DFW, DXB, ZRH",1.66,20.35,78.00,NaN
